# Caso 1:
_Del curso "Python y Machine Learning: de 0 a 100 con Reinforcement Learning"_ <br/>
_Impartido por MalagaAI, [Andrés Matesanz][mate] y [Joaquín Terrasa][quim]_ <br/>
_27 de Marzo de 2020_ <br/>

<small>Capítulo anterior (Introducción a Python): https://github.com/Matesanz/C0-intro-python</small>

[quim]: https://twitter.com/josocjoq
[mate]: https://twitter.com/AIMatesanz

---

### Apliquemos nuestros conocimientos a estudiar el COVID-19 😱🤩

Veremos el caso de España. Para Italia, los datos están [aquí](https://github.com/pcm-dpc/COVID-19).

+ Datos de España *limpiados*: https://github.com/datadista/datasets/tree/master/COVID%2019
+ Datos oficiales: https://covid19.isciii.es/resources/serie_historica_acumulados.csv (dashboard: https://covid19.isciii.es/)

In [2]:
import pandas as pd

df = pd.read_excel("./serie_historica_acumulados.xlsx")
df

,CCAA Codigo ISO,Fecha,Casos,Hospitalizados,UCI,Fallecidos
0,AN,2020-02-20,NaN,NaN,NaN,NaN
1,AR,2020-02-20,NaN,NaN,NaN,NaN
2,AS,2020-02-20,NaN,NaN,NaN,NaN
3,IB,2020-02-20,1.0,NaN,NaN,NaN
4,CN,2020-02-20,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
661,MC,2020-03-25,596.0,172.0,36.0,8.0
662,NC,2020-03-25,1411.0,649.0,54.0,49.0
663,PV,2020-03-25,3946.0,1907.0,137.0,180.0
664,RI,2020-03-25,995.0,316.0,36.0,43.0


In [3]:
df.columns

Index(['CCAA Codigo ISO', 'Fecha', 'Casos ', 'Hospitalizados', 'UCI',
       'Fallecidos'],
      dtype='object')

In [50]:
df = df.drop([665])

---

<div hidden><img src="https://i.imgur.com/9YyVSB2.jpg" width="60%"/></div>

Vamos a visualizar el **impacto del coronavirus en cada CCAA** teniendo en cuenta otros factores, como el ratio de habitantes por CCAA o la riqueza media. Para ello, antes necesitamos un mapa de España:

https://github.com/deldersveld/topojson/#spain

https://geographica.carto.com/u/alasarr/tables/spain_regions/public (esta no la consideramos)

In [6]:
from urllib.request import urlopen
import json

with urlopen("https://raw.githubusercontent.com/deldersveld/topojson/master/countries/spain/spain-comunidad-with-canary-islands.json") as response:
    ccaa = json.load(response)

ccaa.keys()

dict_keys(['type', 'transform', 'arcs', 'objects'])

In [43]:
_ = [(obj["properties"]["NAME_1"], obj.get("properties").get("HASC_1"), obj.get("properties").get("ID_1")) for obj in ccaa["objects"]["ESP_adm1"]["geometries"]]
print(_)
len(_)

[('Andalucía', 'ES.AN', 935), ('Aragón', 'ES.AR', 936), ('Cantabria', 'ES.CB', 937), ('Castilla y León', 'ES.CL', 938), ('Castilla-La Mancha', 'ES.CM', 939), ('Cataluña', 'ES.CT', 940), ('Ceuta y Melilla', 'ES.CE', 941), ('Comunidad de Madrid', 'ES.MD', 942), ('Comunidad Foral de Navarra', 'ES.NA', 943), ('Comunidad Valenciana', 'ES.VC', 944), ('Extremadura', 'ES.EX', 945), ('Galicia', 'ES.GA', 946), ('Islas Baleares', 'ES.PM', 947), ('Islas Canarias', 'ES.CN', 948), ('La Rioja', 'ES.LO', 949), ('País Vasco', 'ES.PV', 950), ('Principado de Asturias', 'ES.AS', 951), ('Región de Murcia', 'ES.MU', 952)]


18

---

Como vemos, en el mapa las comunidades usan el código ISO modificado (`ES.<ISO-CCAA>`), mientras que los datos del COVID19 usan solo el código ISO:

In [45]:
df["CCAA Codigo ISO"].head(3)

0    AN
1    AR
2    AS
Name: CCAA Codigo ISO, dtype: object

In [46]:
df["CCAA Codigo ISO"] = df["CCAA Codigo ISO"].apply(lambda iso: f"ES.{iso}")
df["CCAA Codigo ISO"].head(3)

0    ES.AN
1    ES.AR
2    ES.AS
Name: CCAA Codigo ISO, dtype: object

Hay que comprobar el nº de CCAA en los datos del COVID19:

In [63]:
unique_ccaa_data = list(pd.unique(df["CCAA Codigo ISO"]))
unique_ccaa_data.sort()

print(f"Codigos ISO (datos): ({len(unique_ccaa_data)}) {unique_ccaa_data}")

Codigos ISO (datos): (19) ['ES.AN', 'ES.AR', 'ES.AS', 'ES.CB', 'ES.CE', 'ES.CL', 'ES.CM', 'ES.CN', 'ES.CT', 'ES.EX', 'ES.GA', 'ES.IB', 'ES.MC', 'ES.MD', 'ES.ME', 'ES.NC', 'ES.PV', 'ES.RI', 'ES.VC']


In [64]:
unique_ccaa_mapa = [elem[1] for elem in _]
unique_ccaa_mapa.sort()
print(f"Codigos ISO (mapa): ({len(unique_ccaa_mapa)}) {unique_ccaa_mapa})")

Codigos ISO (mapa): (18) ['ES.AN', 'ES.AR', 'ES.AS', 'ES.CB', 'ES.CE', 'ES.CL', 'ES.CM', 'ES.CN', 'ES.CT', 'ES.EX', 'ES.GA', 'ES.LO', 'ES.MD', 'ES.MU', 'ES.NA', 'ES.PM', 'ES.PV', 'ES.VC'])


[<img src="https://i.imgur.com/4CpleoU.png" width="20%"/><br/><img src="https://i.imgur.com/Cih1Lkp.png" width="16%"/>](https://es.wikipedia.org/wiki/ISO_3166-2:ES)

Por lo tanto, hay que cambiar varios datos ISO del COVID19 (es más facil que cambiar el mapa 😂):

+ Unir las ciudades autónomas de Ceuta (`ES.CE`) y Melilla (`ES.ME`) en el del mapa (`ES.CE`).
+ Cambiar el ISO de Islas Baleares en los datos (ES.IB) al del mapa (ES.PM).
+ Cambiar el ISO de La Rioja en los datos (ES.RI) al del mapa (ES.LR).
+ Cambiar el ISO de Murcia en los datos (ES.MC) al del mapa (ES.MU).
+ Cambiar el ISO de Navarra en los datos (ES.NC) al del mapa (ES.NA).

In [65]:
def renombrar_codigos_iso(codigo_iso):
    """"""
    return {
        "ES.NC": "ES.NA",
        "ES.MC": "ES.MU",
        "ES.RI": "ES.LR",
        "ES.IB": "ES.PM",
        "ES.ME": "ES.CE"
    }.get(codigo_iso, codigo_iso)

df["CCAA Codigo ISO"] = df["CCAA Codigo ISO"].apply(renombrar_codigos_iso)

# tenemos que recordar que, a la hora de extraer los casos por dia y por comunidad, en "Ceuta y Melilla" obtendremos 2 - por lo que hay que sumar!

,CCAA Codigo ISO,Fecha,Casos,Hospitalizados,UCI,Fallecidos
0,ES.AN,2020-02-20,NaN,NaN,NaN,NaN
1,ES.AR,2020-02-20,NaN,NaN,NaN,NaN
2,ES.AS,2020-02-20,NaN,NaN,NaN,NaN


In [66]:
unique_ccaa_data = list(pd.unique(df["CCAA Codigo ISO"]))
unique_ccaa_data.sort()

print(f"Codigos ISO (datos): ({len(unique_ccaa_data)}) {unique_ccaa_data}")

Codigos ISO (datos): (18) ['ES.AN', 'ES.AR', 'ES.AS', 'ES.CB', 'ES.CE', 'ES.CL', 'ES.CM', 'ES.CN', 'ES.CT', 'ES.EX', 'ES.GA', 'ES.LR', 'ES.MD', 'ES.MU', 'ES.NA', 'ES.PM', 'ES.PV', 'ES.VC']


---

Por otra parte, necesitamos los datos del censo más recientes:

https://www.ine.es/jaxiT3/Tabla.htm?t=2915&L=0

Ojo! Fíjate que aquí también se subdividen en 19 CCAA. Ten en cuenta que al incluir acentuaciones, normalmente no usan codificación `UTF-8` 😪😪

In [68]:
censo = pd.read_csv("censo_2019.csv", sep=";", encoding='iso-8859-1')
censo.head(3)

,Comunidades y Ciudades Autónomas,Tamaño de los municipios,Periodo,Total
0,01 Andalucía,Total,2019,8.414.240
1,02 Aragón,Total,2019,1.319.291
2,"03 Asturias, Principado de",Total,2019,1.022.800


In [77]:
list(censo[censo.columns[0]])

['01 Andalucía',
 '02 Aragón',
 '03 Asturias, Principado de',
 '04 Balears, Illes',
 '05 Canarias',
 '06 Cantabria',
 '07 Castilla y León',
 '08 Castilla - La Mancha',
 '09 Cataluña',
 '10 Comunitat Valenciana',
 '11 Extremadura',
 '12 Galicia',
 '13 Madrid, Comunidad de',
 '14 Murcia, Región de',
 '15 Navarra, Comunidad Foral de',
 '16 País Vasco',
 '17 Rioja, La',
 '18 Ceuta',
 '19 Melilla']

Tip: es más facil cambiarlo usando *Excel* o algun programa similar

In [81]:
def obtener_codigos_iso(ccaa):
    """"""
    return {
        "01 Andalucía": 'ES.AN',
        "02 Aragón": 'ES.AR',
        '03 Asturias, Principado de': 'ES.AS',
        '04 Balears, Illes': 'ES.PM',
        '05 Canarias': 'ES.CN',
        '06 Cantabria': 'ES.CB',
        '07 Castilla y León': 'ES.CL',
        '08 Castilla - La Mancha': 'ES.CM',
        '09 Cataluña': 'ES.CT',
        '10 Comunitat Valenciana': 'ES.VC',
        '11 Extremadura': 'ES.EX',
        '12 Galicia': 'ES.GA',
        '13 Madrid, Comunidad de': 'ES.MD',
        '14 Murcia, Región de': 'ES.MU',
        '15 Navarra, Comunidad Foral de': 'ES.NA',
        '16 País Vasco': 'ES.PV',
        '17 Rioja, La': 'ES.LR',
        '18 Ceuta': 'ES.CE',
        '19 Melilla': 'ES.CE'
    }.get(ccaa)

In [82]:
censo["iso"] = censo[censo.columns[0]].apply(obtener_codigos_iso)

In [97]:
censo = censo[["iso", "Total"]]
censo

,iso,Total
0,ES.AN,8.414.240
1,ES.AR,1.319.291
2,ES.AS,1.022.800
3,ES.PM,1.149.460
4,ES.CN,2.153.389
5,ES.CB,581.078
6,ES.CL,2.399.548
7,ES.CM,2.032.863
8,ES.CT,7.675.217
9,ES.VC,5.003.769


In [99]:
censo.groupby("iso").sum()

,Total
iso,
ES.AN,8.414.240
ES.AR,1.319.291
ES.AS,1.022.800
ES.CB,581.078
ES.CE,84.77786.487
ES.CL,2.399.548
ES.CM,2.032.863
ES.CN,2.153.389
ES.CT,7.675.217


In [104]:
censo["Total"].apply(lambda num: num.replace(".", "")).astype(int)

0     8414240
1     1319291
2     1022800
3     1149460
4     2153389
5      581078
6     2399548
7     2032863
8     7675217
9     5003769
10    1067710
11    2699499
12    6663394
13    1493898
14     654214
15    2207776
16     316798
17      84777
18      86487
Name: Total, dtype: int32

In [105]:
censo["Total"] = censo["Total"].apply(lambda num: num.replace(".", "")).astype(int)
censo = censo.groupby("iso").sum()
censo

c:\users\pachacho\miniconda3\envs\pycourse\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Total
iso,
ES.AN,8414240
ES.AR,1319291
ES.AS,1022800
ES.CB,581078
ES.CE,171264
ES.CL,2399548
ES.CM,2032863
ES.CN,2153389
ES.CT,7675217


In [109]:
print(censo.index)
print(censo.dtypes)

censo["iso"] = censo.index

Index(['ES.AN', 'ES.AR', 'ES.AS', 'ES.CB', 'ES.CE', 'ES.CL', 'ES.CM', 'ES.CN',
       'ES.CT', 'ES.EX', 'ES.GA', 'ES.LR', 'ES.MD', 'ES.MU', 'ES.NA', 'ES.PM',
       'ES.PV', 'ES.VC'],
      dtype='object', name='iso')
Total    int32
dtype: object


In [110]:
ls = list(censo["iso"])
print(f"Codigos ISO (censo): ({len(ls)}) {ls}")

Codigos ISO (censo): (18) ['ES.AN', 'ES.AR', 'ES.AS', 'ES.CB', 'ES.CE', 'ES.CL', 'ES.CM', 'ES.CN', 'ES.CT', 'ES.EX', 'ES.GA', 'ES.LR', 'ES.MD', 'ES.MU', 'ES.NA', 'ES.PM', 'ES.PV', 'ES.VC']


---

Por último, podemos juntar todos estos datos y hacer un bonito mapa 😍

In [111]:
# datos del censo 2019
censo.head(2)

,Total,iso
iso,,
ES.AN,8414240,ES.AN
ES.AR,1319291,ES.AR


In [112]:
# serie temporal COVID19
df.head(2)

,CCAA Codigo ISO,Fecha,Casos,Hospitalizados,UCI,Fallecidos
0,ES.AN,2020-02-20,NaN,NaN,NaN,NaN
1,ES.AR,2020-02-20,NaN,NaN,NaN,NaN


In [114]:
# mapa spain
ccaa.keys()

dict_keys(['type', 'transform', 'arcs', 'objects'])

**Primero**, crearemos un mapa estático de la población por comunidades. Y **segundo**, crearemos una animación que permita visualizar la serie temporal.

In [ ]:
import plotly.express as px



In [17]:
x = df.iloc[665:666, 0]
x

665    NOTA: El objetivo de los datos que se publican...
Name: CCAA Codigo ISO, dtype: object

In [33]:
x.to_dict()[665]

'NOTA: El objetivo de los datos que se publican en esta web es saber el n�mero de casos acumulados a la fecha y que por tanto no se puede deducir que la diferencia entre un d�a y el anterior es el n�mero de casos nuevos ya que esos casos pueden haber sido recuperados de fechas anteriores. Cualquier inferencia que se haga sobre las diferencias de un d�a para otro deben hacerse con precauci�n y son �nicamente la responsabilidad el autor.'

---

## ¿Más dudas?

Puedes encontrarnos por

+ Twitter: en @AIMalaga, @AIMatesanz (Andrés) y @josocjoq (Joaquín)
+ Telegram: en nuestro grupo https://t.me/MalagaSchoolOfAI
> Ojito! Este es el grupo de MálagaAI. Si perteneces a otra ciudad, quizás te interesa [encontrar tu comunidad local](https://www.spain-ai.com/). Si quieres crear una comunidad [contacta con SpainAI en Twitter](https://twitter.com/Spain_AI_).
+ Linkedin: [Spain AI](https://www.linkedin.com/company/spainai/)

---

Licencia: Andrés Matesanz y Joaquín Terrasa, MalagaAI [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/) 2020

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Cc.logo.circle.svg/64px-Cc.logo.circle.svg.png" width="1%"/><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Cc-by_new.svg/80px-Cc-by_new.svg.png" width="1%"/><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Cc-nc.svg/64px-Cc-nc.svg.png" width="1%"/>